In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import os
import openslide
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device('cpu')

In [56]:
metadata_path = '/scratch/cz2064/myjupyter/BDML/Project/metadata/gdc_sample_sheet.2020-03-28.tsv'
metadata = pd.read_csv(metadata_path, sep='\t')
metadata['type'] = 'Normal'
metadata.loc[metadata['Sample Type']!='Solid Tissue Normal','type']='Tumor-LUSC'
metadata.loc[(metadata['Sample Type']!='Solid Tissue Normal') & (metadata['Project ID']=='TCGA-LUAD'),\
             'type']='Tumor-LUAD'
metadata['label'] = 0
metadata.loc[metadata['type']=='Tumor-LUSC','label'] = 1
metadata.loc[metadata['type']=='Tumor-LUAD','label'] = 2
metadata.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,label
0,e9760244-3db6-48e1-aa57-f2e2b5352934,TCGA-18-3419-01A-01-BS1.258a5fb6-f212-4a6a-a43...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-18-3419,TCGA-18-3419-01A,Primary Tumor,Tumor-LUSC,1
1,c9b69594-7be5-4597-8698-7c2880b14b24,TCGA-56-8305-11A-01-TS1.dc4b057e-a254-4147-84c...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-56-8305,TCGA-56-8305-11A,Solid Tissue Normal,Normal,0
2,79ec2dcc-5ca0-4651-aeac-26db87d4f696,TCGA-34-2600-11A-01-BS1.dd70f786-4ac1-4dfc-b66...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-34-2600,TCGA-34-2600-11A,Solid Tissue Normal,Normal,0
3,70d0010d-8346-4d48-8620-16bbe3523667,TCGA-77-A5G6-01A-01-TS1.694AEB4E-FE41-4226-899...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-77-A5G6,TCGA-77-A5G6-01A,Primary Tumor,Tumor-LUSC,1
4,a369fc28-650b-4587-83dc-78fa8875ef16,TCGA-56-A62T-01A-01-TS1.E3B64735-90EE-436F-901...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-56-A62T,TCGA-56-A62T-01A,Primary Tumor,Tumor-LUSC,1


In [58]:
idx = np.arange(len(metadata))
np.random.seed(0)
np.random.shuffle(idx)
val_size = int(len(metadata)/5) # 20% validation
test_size = int(len(metadata)/5) # 20% test
train_size = len(metadata) - val_size - test_size
train_idx = idx[:train_size]
val_idx = idx[train_size:train_size+val_size]
test_idx = idx[train_size+val_size:]
train_df = metadata.iloc[train_idx,:]
val_df = metadata.iloc[val_idx,:]
test_df = metadata.iloc[test_idx,:]
train_df.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,label
317,0242c352-73ec-477e-b6e7-0a4283f0e981,TCGA-L9-A743-01A-04-TS4.873E50BA-0B07-4174-9FB...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-L9-A743,TCGA-L9-A743-01A,Primary Tumor,Tumor-LUAD,2
506,7888cbbc-6a44-4d76-ab8a-c4f8a8913354,TCGA-78-7162-01A-01-BS1.c4ea5644-a749-456e-a83...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-78-7162,TCGA-78-7162-01A,Primary Tumor,Tumor-LUAD,2
161,1617b5f5-7a9c-4657-97a3-3b4cb0ace72a,TCGA-39-5021-01A-01-BS1.b64b9118-29ed-4f26-a24...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-39-5021,TCGA-39-5021-01A,Primary Tumor,Tumor-LUSC,1
1051,b0a5fe8b-65f7-4a85-adb1-3b2ab4180d65,TCGA-L9-A7SV-01A-01-TS1.E157B486-A95D-4D6F-997...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-L9-A7SV,TCGA-L9-A7SV-01A,Primary Tumor,Tumor-LUAD,2
2070,a7b5baa5-3c69-40d1-aefd-b47129546109,TCGA-66-2765-11A-01-BS1.03683356-18f2-4deb-a6f...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-66-2765,TCGA-66-2765-11A,Solid Tissue Normal,Normal,0


In [59]:
train_df.to_csv('train.csv', index = False)
val_df.to_csv('val.csv', index = False)
test_df.to_csv('test.csv', index = False)